# Suite: Sales
Using Library Python Selenium to test the web site

In [1]:
# ####### DEBUG ONLY #######
# ####### Load the "autoreload" extension #######
%load_ext autoreload
# ####### always reload modules marked with "%import" #######
%autoreload 2

import os
import sys

sys.path.append("../config")
sys.path.append("../util")
sys.path.append("../tests")
sys.path.append("../../library/util")

In [2]:
# Importing packages
from config import config
from datetime import datetime, timedelta
import json
import logging
import logging.config
import pathlib
from pprint import pprint
import web_driver
from web_driver import WebDriver
import time

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys

import unittest
from ui_helper import UIHelper

## Variables Declaration

In [3]:
log_config = "../config/logging.conf"

# # Create folder "logs"
# pathlib.Path('../logs').mkdir(exist_ok=True)  
# logging.log_file = datetime.now().strftime('../logs/log_%Y%m%d_%H%M%S.log')
logging.config.fileConfig(log_config, disable_existing_loggers=False)
logger = logging.getLogger(__name__)

suite_name = "suite_sales"
suite_case = "suite_02"
logger.info("Test Suite: {}".format(suite_name))
logger.info("Suite Case: {}".format(suite_case))

[INFO] Test Suite: suite_sales
[INFO] Suite Case: suite_02


## Functions Definition

### Main Function Begin
def main(args):

In [4]:
mode = config["web"]["mode"]
base_url = config["web"]["base_url"]
timeout = config["web"]["timeout"]
user_name = config["web"]["user_name"]
password = config["web"]["password"]
company_short_name = config["data"]["company_short_name"]
company_name = config["data"]["company_name"]

if mode == "windows":
    driver_path = "../../library/drivers/chrome/win/chromedriver.exe"
else:
    driver_path = "../../library/drivers/chrome/mac/chromedriver"
    
logger.info("Mode: {}".format(mode))
logger.info("Base Url: {}".format(base_url))
logger.info("User Name: {}".format(user_name))

[INFO] Mode: windows
[INFO] Base Url: http://211.25.204.108:30002
[INFO] User Name: zong.xian.soh@adv-fusionex.com


In [5]:
# Get web page
instance = WebDriver(mode="windows", driver_path=driver_path, timeout=timeout)
instance.get_driver().get(base_url)

WebDriver created!
Mode: windows
Driver Path: ../../library/drivers/chrome/win/chromedriver.exe
Timeout: 10


<a id='top'></a>
## Table of contents
- <a href='#data_load'>Load Test Data</a>
- <a href='#login'>Login</a>
- <a href='#switch_company'>Switch Company</a>
- <a href='#invoices'>Sales - Invoices</a>
- <a href='#revenues'>Sales - Revenues</a>
- <a href='#credit_notes'>Sales - Credit Notes</a>
- <a href='#customers'>Customers</a>
- <a href='#collections'>Collections</a>

<a id='login'></a>
### Login
<a href='#top'>top</a>

In [8]:
ui = UIHelper(instance)
ui.login(user_name=user_name, password=password)

<a id='data_load'></a>
### Load Test Data
<a href='#top'>top</a>

In [ ]:
# Load sample data
data_path =  f"../data/{suite_case}/sales.json"
data = ui.load_data(data_path)


invoices = data["invoices"]
revenues = data["revenues"]
credit_notes = data["credit_notes"]
customers = data["customers"]
collections = data["collections"]

<a id='switch_company'></a>
### Switch Company
<a href='#top'>top</a>

In [ ]:
ui.switch_company(company_name)

<a id='invoices'></a>
### Sales - Invoices
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Sales", "Invoices")

In [ ]:
for invoice in invoices:
    ui.click_add()
    ui.select_dropdown("Customer", invoice["customer"])
    ui.select_dropdown("Currency", invoice["currency"])
    ui.select_date_picker("Invoice Date", invoice["invoiced_at"])
    ui.enter_text((By.ID, "invoice_number"), invoice["invoice_number"])
    ui.enter_text((By.ID, "order_number"), invoice["order_number"])
    
    locator = (By.XPATH, '//div[@class="card-body"]//div[contains(@class, "col-sm-12 mb-4") and .//*[text()[contains(., "Items")]]]')
    elements = instance.find_elements(locator)
    # to make Items visible
    instance.scroll_to_element(elements[0])
    for i in range(len(invoice["items"])):
        account_type = invoice["items"][i]["account_type"]
        name = invoice["items"][i]["name"]
        quantity = invoice["items"][i]["quantity"]
        # Select Account Type
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]/td[2]//span[@class="el-input__prefix"]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{account_type}"]]') 
        instance.click_element(locator)
        # Select Name
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]/td[3]//span[@class="el-input__prefix"]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{name}"]]') 
        instance.click_element(locator)
        # Insert Quantity
        ui.enter_text((By.NAME, f"items.{i}.quantity"), str(quantity))
    # Add discount
    instance.click_element((By.XPATH, './/tr[@id="tr-discount"]/td/span//span[@class="el-link--inner"]'))
    instance.enter_text((By.XPATH, './/input[@id="pre-discount"]'), invoice["discount_rate"])
    instance.click_element((By.XPATH, './/button[@id="save-discount"]'))
    instance.scroll_down()
    # Select Category
    ui.select_dropdown("Category", invoice["category"])
    ui.click_save()
    instance.wait(5)
    if invoice["status"] == "paid":
        # Add payment between here
        instance.click_element((By.ID, "button-payment"))
        instance.wait(2)
        instance.click_element((By.XPATH, f'.//div[@class="row"]//div[contains(@class, "form-group") and .//*[text()[contains(., "Account")]]]'))
        instance.wait(2)
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{invoice["account"]}"]]')
        instance.click_element(locator)
        instance.wait(3)
        instance.click_element((By.XPATH, f'.//div[@class="row"]//div[contains(@class, "form-group") and .//*[text()[contains(., "Payment Method")]]]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{invoice["payment"]}"]]') 
        instance.click_element(locator)
        ui.enter_text((By.NAME, "description"), invoice["description"])
        ui.enter_text((By.ID, "reference"), invoice["reference"])
        locator = (By.XPATH, './/button[contains(@class, "btn button-submit btn-success")]')
        instance.click_element(locator)
        instance.wait(5)
    else:
        instance.click_element((By.XPATH, f'//div[@class="mt-3"]//*[text()[contains(., "Mark Sent")]]'))
        instance.wait(2)
        
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Invoices")]]'))
    instance.wait(2)

<a id='revenues'></a>
### Sales - Revenues
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Sales", "Revenues")

In [ ]:
for revenue in revenues:
    ui.click_add()
    ui.select_date_picker("Date", revenue["date"])
    ui.enter_text((By.NAME, "amount"), str(revenue["amount"]) + "0000")
    ui.select_dropdown("Account", revenue["account"])
    ui.select_dropdown("Customer", revenue["customer"])
    instance.scroll_down()
    ui.select_dropdown("Category", revenue["category"])
    ui.select_dropdown("Recurring", revenue["recurring"])
    ui.select_dropdown("Payment Method", revenue["payment_method"])
    
    ui.click_save()
    instance.wait(5)

<a id='credit_notes'></a>
### Credit Notes
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Sales", "Credit Notes")

In [ ]:
for credit_note in credit_notes:
    ui.click_add()
    instance.wait(2)
    ui.select_dropdown("Customer", credit_note["customer"])
    ui.select_dropdown("Currency", credit_note["currency"])
    ui.select_date_picker("Credit Note Date", credit_note["credit_at"])
    ui.enter_text((By.ID, "credit_note_number"), credit_note["credit_note_number"])
    ui.select_dropdown("Invoice", credit_note["invoice_number"])

    locator = (By.XPATH, '//div[@class="card-body"]//div[contains(@class, "col-sm-12 mb-4") and .//*[text()[contains(., "Items")]]]')
    elements = instance.find_elements(locator)
    # to make Items visible
    instance.scroll_to_element(elements[0])
    for i in range(len(credit_note["items"])):
        name = credit_note["items"][i]["name"]
        quantity = credit_note["items"][i]["quantity"]
        # Select Name
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]/td[2]//span[@class="el-input__prefix"]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{name}"]]') 
        instance.click_element(locator)
        # Insert Quantity
        ui.enter_text((By.NAME, f"items.{i}.quantity"), str(quantity))
    # Add Discount
    instance.click_element((By.XPATH, './/tr[@id="tr-discount"]/td/span//span[@class="el-link--inner"]'))
    instance.enter_text((By.XPATH, './/input[@id="pre-discount"]'), credit_note["discount_rate"])
    instance.click_element((By.XPATH, './/button[@id="save-discount"]'))
    instance.scroll_down()
    # Select Category
    ui.select_dropdown("Category", credit_note["category"])
    ui.click_save()
    instance.wait(5)
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Credit Notes")]]'))
    instance.wait(2)

<a id='customers'></a>
### Sales - Customers
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Sales", "Customers")

In [ ]:
for customer in customers:
    locator = (By.XPATH, f'.//tbody/tr[contains(@class, "row") and .//a[contains(text(), "{customer["name"]}")]]')
    elements = instance.find_elements(locator)
    
    if len(elements) > 0:
        logger.info(f"Found: {customer['name']}")
        instance.scroll_to_element(elements[0])
        ui.click_edit(elements[0])
    else:
        logger.info(f"Not Found: {customer['name']}")
        ui.click_add()

    ui.enter_text((By.ID, "name"), customer["name"])
    ui.enter_text((By.ID, "email"), customer["email"])
    ui.enter_text((By.ID, "tax_number"), customer["tax"])
    ui.select_dropdown("Currency", customer["currency"])
    ui.enter_text((By.ID, "phone"), customer["phone"])
    ui.enter_text((By.ID, "website"), customer["website"])
    ui.enter_text((By.ID, "address"), customer["address"])
    ui.enter_text((By.ID, "reference"), customer["reference"])
    instance.scroll_down()
    ui.click_save()
    instance.wait(5)
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Customers")]]'))
    instance.wait(2)

In [ ]:
instance.close()

<a id='collections'></a>
### Sales - Collections
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Sales", "Collections")

In [ ]:
for collection in collections:
    ui.click_add()
    instance.wait(2)
    ui.select_date_picker("Date", collection["date"])
    ui.enter_text((By.NAME, "amount"), str(collection["amount"]))
    ui.select_dropdown("Account", collection["account"])
    ui.select_dropdown("Customer", collection["customer"])
    ui.enter_text((By.NAME, "description"), collection["description"])
    instance.scroll_down()
    ui.select_dropdown("Category", collection["category"])
    ui.select_dropdown("Payment Method", collection["payment"])
    ui.enter_text((By.ID, "reference"), collection["reference"])
    ui.click_save()
    instance.wait(5)
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Collections")]]'))
    instance.wait(2)

In [ ]:
instance.close()